# 캐글 타이타닉 튜토리얼 따라하기
참고 : https://www.kaggle.com/datacanary/xgboost-example-python

## 데이터 로드
* 업로드를 사용하지 않고 판다스에 url을 지정해서 가져올 수도 있지만 캐글 데이터는 경진대회의 규칙에 동의했을 때에만 다운로드가 가능해서 그냥 불러오면 SSL 오류가 발생한다. 그래서 불편하지만 로컬 파일을 업로드 하는 방법으로 데이터를 읽어오도록 했다.
* 데이터 업로드는 colaboratory 왼쪽 탭의 code snippets를 참고했다.

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
import pandas as pd
import io

train_df = pd.read_csv(
    io.StringIO(uploaded['train.csv'].decode('utf-8')), header=0)

test_df = pd.read_csv(
    io.StringIO(uploaded['test.csv'].decode('utf-8')), header=0)

### 로드된 데이터 확인

In [ ]:
print(train_df.shape)
print(test_df.shape)

In [ ]:
train_df.head()

In [ ]:
test_df.head()

## 피처 생성

* 결측치 보완 

In [ ]:
# 누락된 데이터를 중간값으로 채워준다.
# This is based on some nice code by 'sveitser' at http://stackoverflow.com/a/25562948
from sklearn.base import TransformerMixin
import numpy as np
from sklearn.preprocessing import LabelEncoder

class DataFrameImputer(TransformerMixin):
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].median() for c in X],
            index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.fill)

# 피처로 사용할 컬럼      
feature_columns_to_use = ['Pclass','Sex','Age','Fare','Parch']
# 카테고리 데이터로 숫자로 변환해 주기 위해 따로 담아준다.
nonnumeric_columns = ['Sex']

In [ ]:
# Join the features from train and test together before imputing missing values,
# in case their distribution is slightly different
# 누락된 데이터를 채워주기 전에 train, test 데이터를 합쳐준다.
# 두 데이터의 분포가 약간 다르다.
big_X = train_df[feature_columns_to_use].append(test_df[feature_columns_to_use])
big_X_imputed = DataFrameImputer().fit_transform(big_X)

In [ ]:
# XGBoost는 카테고리 피처를 다룰 수 없기 때문에 숫자로 변환해 준다.
# 사이킷런 문서에서 자세한 사용법을 알 수 있다.
# See http://scikit-learn.org/stable/modules/preprocessing.html#preprocessing for more

le = LabelEncoder()
for feature in nonnumeric_columns:
    big_X_imputed[feature] = le.fit_transform(big_X_imputed[feature])

In [ ]:
# Prepare the inputs for the model
train_X = big_X_imputed[0:train_df.shape[0]].as_matrix()
test_X = big_X_imputed[train_df.shape[0]::].as_matrix()
train_y = train_df['Survived']

In [ ]:
# https://github.com/dmlc/xgboost
# 새로운 버전에는 빌드이슈가 있기 때문에 특정 버전을 설치해 준다.
!pip install -q xgboost==0.4a30
import xgboost as xgb

## 학습

In [ ]:
# You can experiment with many other options here, using the same .fit() and .predict()
# methods; see http://scikit-learn.org
# This example uses the current build of XGBoost, from https://github.com/dmlc/xgboost

params = {
    'booster': 'gblinear',
    'objective': 'multi:softmax',
#     'objective': 'binary:logistic',
    'eval_metric': 'merror',
    'lambda': 2.0,
    'alpha': 1.0,
    'lambda_bias': 6.0,
    'num_class': 5,
#     'learning_rate': 0.05,
    'nthread': 8,
    'n_jobs': -1,
    'silent': 1,
    'n_estimators' : 300,
#     'max_depth' : 3, 
}
xgb.XGBClassifier(params)

gbm = xgb.XGBClassifier(params)
gbm

## 평가 측정

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

k_fold = KFold(n_splits=10, shuffle=True, random_state=2018)

%time score = cross_val_score(gbm, train_X, train_y, cv=k_fold, n_jobs=-1, scoring="accuracy").mean()

print("Score = {0:.5f}".format(score))

## 예측

In [ ]:
%time gbm.fit(train_X, train_y)
%time predictions = gbm.predict(test_X)

## 제출

In [ ]:
# Kaggle needs the submission to have a certain format;
# see https://www.kaggle.com/c/titanic-gettingStarted/download/gendermodel.csv
# for an example of what it's supposed to look like.
submission = pd.DataFrame({ 'PassengerId': test_df['PassengerId'],
                            'Survived': predictions })
submission.head()

In [ ]:
submission['Survived'].value_counts()

In [ ]:
submission.to_csv("submission_{0:.5f}.csv".format(score), index=False)

In [ ]:
# 왼쪽 Code snippets의 로컬 파일시스템으로 다운로드하기를 참고한다.
files.download("submission_{0:.5f}.csv".format(score))